<a href="https://colab.research.google.com/github/TimTree/DS-Unit-2-Kaggle-Challenge/blob/master/assignment_kaggle_challenge_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Kaggle Challenge, Module 4

## Assignment
- [ ] If you haven't yet, [review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Plot a confusion matrix for your Tanzania Waterpumps model.
- [ ] Continue to participate in our Kaggle challenge. Every student should have made at least one submission that scores at least 60% accuracy (above the majority class baseline).
- [ ] Submit your final predictions to our Kaggle competition. Optionally, go to **My Submissions**, and _"you may select up to 1 submission to be used to count towards your final leaderboard score."_
- [ ] Commit your notebook to your fork of the GitHub repo.
- [ ] Read [Maximizing Scarce Maintenance Resources with Data: Applying predictive modeling, precision at k, and clustering to optimize impact](https://towardsdatascience.com/maximizing-scarce-maintenance-resources-with-data-8f3491133050), by Lambda DS3 student Michael Brady. His blog post extends the Tanzania Waterpumps scenario, far beyond what's in the lecture notebook.


## Stretch Goals

### Reading
- [Attacking discrimination with smarter machine learning](https://research.google.com/bigpicture/attacking-discrimination-in-ml/), by Google Research, with  interactive visualizations. _"A threshold classifier essentially makes a yes/no decision, putting things in one category or another. We look at how these classifiers work, ways they can potentially be unfair, and how you might turn an unfair classifier into a fairer one. As an illustrative example, we focus on loan granting scenarios where a bank may grant or deny a loan based on a single, automatically computed number such as a credit score."_
- [Notebook about how to calculate expected value from a confusion matrix by treating it as a cost-benefit matrix](https://github.com/podopie/DAT18NYC/blob/master/classes/13-expected_value_cost_benefit_analysis.ipynb)
- [Simple guide to confusion matrix terminology](https://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/) by Kevin Markham, with video
- [Visualizing Machine Learning Thresholds to Make Better Business Decisions](https://blog.insightdatascience.com/visualizing-machine-learning-thresholds-to-make-better-business-decisions-4ab07f823415)


### Doing
- [ ] Share visualizations in our Slack channel!
- [ ] RandomizedSearchCV / GridSearchCV, for model selection. (See below)
- [ ] Stacking Ensemble. (See below)
- [ ] More Categorical Encoding. (See below)

### RandomizedSearchCV / GridSearchCV, for model selection

- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?

### Stacking Ensemble

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```


### More Categorical Encodings

**1.** The article **[Categorical Features and Encoding in Decision Trees](https://medium.com/data-design/visiting-categorical-features-and-encoding-in-decision-trees-53400fa65931)** mentions 4 encodings:

- **"Categorical Encoding":** This means using the raw categorical values as-is, not encoded. Scikit-learn doesn't support this, but some tree algorithm implementations do. For example, [Catboost](https://catboost.ai/), or R's [rpart](https://cran.r-project.org/web/packages/rpart/index.html) package.
- **Numeric Encoding:** Synonymous with Label Encoding, or "Ordinal" Encoding with random order. We can use [category_encoders.OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html).
- **One-Hot Encoding:** We can use [category_encoders.OneHotEncoder](http://contrib.scikit-learn.org/categorical-encoding/onehot.html).
- **Binary Encoding:** We can use [category_encoders.BinaryEncoder](http://contrib.scikit-learn.org/categorical-encoding/binary.html).


**2.** The short video 
**[Coursera — How to Win a Data Science Competition: Learn from Top Kagglers — Concept of mean encoding](https://www.coursera.org/lecture/competitive-data-science/concept-of-mean-encoding-b5Gxv)** introduces an interesting idea: use both X _and_ y to encode categoricals.

Category Encoders has multiple implementations of this general concept:

- [CatBoost Encoder](http://contrib.scikit-learn.org/categorical-encoding/catboost.html)
- [James-Stein Encoder](http://contrib.scikit-learn.org/categorical-encoding/jamesstein.html)
- [Leave One Out](http://contrib.scikit-learn.org/categorical-encoding/leaveoneout.html)
- [M-estimate](http://contrib.scikit-learn.org/categorical-encoding/mestimate.html)
- [Target Encoder](http://contrib.scikit-learn.org/categorical-encoding/targetencoder.html)
- [Weight of Evidence](http://contrib.scikit-learn.org/categorical-encoding/woe.html)

Category Encoder's mean encoding implementations work for regression problems or binary classification problems. 

For multi-class classification problems, you will need to temporarily reformulate it as binary classification. For example:

```python
encoder = ce.TargetEncoder(min_samples_leaf=..., smoothing=...) # Both parameters > 1 to avoid overfitting
X_train_encoded = encoder.fit_transform(X_train, y_train=='functional')
X_val_encoded = encoder.transform(X_train, y_val=='functional')
```

**3.** The **[dirty_cat](https://dirty-cat.github.io/stable/)** library has a Target Encoder implementation that works with multi-class classification.

```python
 dirty_cat.TargetEncoder(clf_type='multiclass-clf')
```
It also implements an interesting idea called ["Similarity Encoder" for dirty categories](https://www.slideshare.net/GaelVaroquaux/machine-learning-on-non-curated-data-154905090).

However, it seems like dirty_cat doesn't handle missing values or unknown categories as well as category_encoders does. And you may need to use it with one column at a time, instead of with your whole dataframe.

**4. [Embeddings](https://www.kaggle.com/learn/embeddings)** can work well with sparse / high cardinality categoricals.

_**I hope it’s not too frustrating or confusing that there’s not one “canonical” way to encode categorcals. It’s an active area of research and experimentation! Maybe you can make your own contributions!**_

In [1]:
import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('/content')

Reinitialized existing Git repository in /content/.git/
fatal: remote origin already exists.
From https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge
 * branch            master     -> FETCH_HEAD
Already up to date.


In [2]:
!ls

data		 LICENSE  module2  module4    requirements.txt
environment.yml  module1  module3  README.md  sample_data


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.merge(pd.read_csv('data/waterpumps/train_features.csv'), 
                 pd.read_csv('data/waterpumps/train_labels.csv'))
test = pd.read_csv('data/waterpumps/test_features.csv')
sample_submission = pd.read_csv('data/waterpumps/sample_submission.csv')

train.shape, test.shape

((59400, 41), (14358, 40))

In [0]:
from sklearn.model_selection import train_test_split
train, validate = train_test_split(train, random_state=9809)

In [0]:
import numpy as np

def wrangle(X):
  # Wrangle the train, validate, and test datasets in one function

  # Prevent SettingWithCopyWarning
  X = X.copy()

  # About 3% of the time, latitude has small values near zero,
  # outside Tanzania, so we'll treat these values like zero.
  X['latitude'] = X['latitude'].replace(-2e-08, 0)

  # There are some values labeled as 0 that should be null values
  # (ex: Construction year can't realistically be 0). Let's convet
  # such values into nulls.
  cols_with_zeros = ['longitude', 'latitude', 'construction_year','population','amount_tsh','gps_height']
  for col in cols_with_zeros:
    X[col] = X[col].replace(0, np.nan)
  
  # drop duplicate columns
  X = X.drop(columns=['quantity_group', 'payment_type'])
  
  # Generate column that gives time difference from construction to inspection
  X['construction_to_inspection'] = pd.to_datetime(X['date_recorded']).dt.year - X['construction_year']

  # return the wrangled dataframe
  return X

In [0]:
train = wrangle(train)
validate = wrangle(validate)
test = wrangle(test)

In [7]:
train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,water_quality,quality_group,quantity,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group,construction_to_inspection
40513,63852,NaN,2012-11-13,Biore,NaN,BioRe,34.096062,-3.413126,Sangamwalugesha Primary School,0,Internal,Magharibi,Shinyanga,17,2,Maswa,Dakama,NaN,True,GeoData Consultants Ltd,WUG,NaN,False,NaN,gravity,gravity,gravity,wug,user-group,other,soft,good,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional,NaN
20837,30885,NaN,2011-07-18,World Bank,1150.0,DWE,31.225618,-6.322244,Mwenge Shule Ya Msingi,0,Lake Rukwa,Mwenge Shule Ya Msingi,Rukwa,15,1,Mpanda,Mtapenda,400.0,False,GeoData Consultants Ltd,Water authority,Kenswa,True,2008.0,gravity,gravity,gravity,water authority,commercial,never pay,soft,good,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional needs repair,3.0
2719,61733,NaN,2013-01-04,Government Of Tanzania,1269.0,Government,37.572751,-3.298947,Kwa Stanley Makata,0,Pangani,Sokony,Kilimanjaro,3,4,Moshi Rural,Mwika Kusini,1.0,True,GeoData Consultants Ltd,WUA,Marera-Lole pipeline,True,2003.0,gravity,gravity,gravity,wua,user-group,pay monthly,soft,good,insufficient,spring,spring,groundwater,communal standpipe multiple,communal standpipe,functional needs repair,10.0
28051,7805,20.0,2011-03-09,Yaole,73.0,DWE,38.664807,-5.804634,Kwa Yaole,0,Pangani,Sokoni,Tanga,4,5,Pangani,Mkalamo,20.0,False,GeoData Consultants Ltd,Private operator,Mkalamo water supply,False,2009.0,submersible,submersible,submersible,private operator,commercial,pay per bucket,salty,salty,insufficient,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,non functional,2.0
5232,47590,500.0,2011-03-26,Caritas,322.0,DWE,36.406274,-8.671257,Kwa Gwawa,0,Rufiji,Ibuta,Morogoro,5,4,Ulanga,Itete,1.0,True,GeoData Consultants Ltd,Water Board,Itete wa,True,2006.0,gravity,gravity,gravity,vwc,user-group,pay monthly,soft,good,enough,river,river/lake,surface,communal standpipe,communal standpipe,functional,5.0


In [8]:
# The target is the the status group
target = 'status_group'

# There's a lot of features here, numeric and categorical.

# Begin by getting all the features that are not the ID or status_group
train_features = train.drop(columns=[target,'id'])

# Let's take in all the numeric features, which is the all of them minus
# the target (and id in this case).
numerics = train_features.select_dtypes(include='number').columns.tolist()

# And here's all the categorical features
categoricals = train_features.select_dtypes(exclude='number').columns.tolist()

# Some categorical variables may have a ton of unique values. Not only will
# so many unique values make our model difficult to generalize, we'll overflow
# our computer's RAM if we did a one-hot encode of them (will explain one-hot
# encode shortly)
# So let's only accept low cardinality categoricals to analyze (that is, in
# this case, the categorical variables with 21 or less unique values.)
low_cardinality_categoricals = [col for col in categoricals
                               if train_features[col].nunique() <= 21]

# Now here are our features.
features = numerics + low_cardinality_categoricals
print(features)

['amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private', 'region_code', 'district_code', 'population', 'construction_year', 'construction_to_inspection', 'basin', 'region', 'public_meeting', 'recorded_by', 'scheme_management', 'permit', 'extraction_type', 'extraction_type_group', 'extraction_type_class', 'management', 'management_group', 'payment', 'water_quality', 'quality_group', 'quantity', 'source', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group']


In [0]:
# Override features in attempt to improve validation score
features = train_features.select_dtypes(include='number').columns.tolist() + train_features.select_dtypes(exclude='number').columns.tolist()

In [0]:
X_train = train[features]
y_train = train[target]
X_validate = validate[features]
y_validate = validate[target]
X_test = test[features]

In [11]:
%%time

import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import f_regression, SelectKBest, f_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='mean'),
    RandomForestClassifier(n_estimators=100,random_state=235,n_jobs=-1)
)

# Fit on train
pipeline.fit(X_train, y_train)

# Score on validate
print('Validation Accuracy', pipeline.score(X_validate, y_validate))

# Make a prediction
y_pred = pipeline.predict(X_validate)

Validation Accuracy 0.8101683501683502
CPU times: user 19.6 s, sys: 210 ms, total: 19.9 s
Wall time: 10.9 s


In [12]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_validate, y_pred)

array([[7213,  190,  711],
       [ 525,  367,  131],
       [1181,   81, 4451]])

In [13]:
from sklearn.utils.multiclass import unique_labels

unique_labels(y_validate)

array(['functional', 'functional needs repair', 'non functional'],
      dtype='<U23')

In [14]:
def plot_confusion_matrix(y_true, y_pred):
    labels = unique_labels(y_true)
    columns = ['Predicted ' + str(label) for label in labels]
    index = ['Actual ' + str(label) for label in labels]
    table = pd.DataFrame(confusion_matrix(y_true, y_pred), 
                         columns=columns, index=index)
    return table

plot_confusion_matrix(y_validate, y_pred)

,Predicted functional,Predicted functional needs repair,Predicted non functional
Actual functional,7213,190,711
Actual functional needs repair,525,367,131
Actual non functional,1181,81,4451


In [15]:
from sklearn.metrics import classification_report
print(classification_report(y_validate,y_pred))

                         precision    recall  f1-score   support

             functional       0.81      0.89      0.85      8114
functional needs repair       0.58      0.36      0.44      1023
         non functional       0.84      0.78      0.81      5713

               accuracy                           0.81     14850
              macro avg       0.74      0.68      0.70     14850
           weighted avg       0.81      0.81      0.80     14850



In [21]:
# Precision (Functional): 7213 / (7213 + 525 + 1181)

# The number of times we predicted functional and it is functional over all guesses in which the answer was actually functional

7213/(7213+525+1181)

0.8087229510034757

In [22]:
# Recall (Functional): 7213 / (7213 + 190 + 711)

# The number of times we predicted functional and it is functional over the sum of all our predicted functionals.

7213/(7213+190+711)

0.8889573576534385

In [0]:
y_predTest = pipeline.predict(X_test)

In [18]:
submission = test[['id']].copy()
submission['status_group'] = y_predTest
submission.head()

,id,status_group
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional


In [0]:
submission.to_csv('kaggleChallenge.csv', index=False)

In [20]:
ls

data/                LICENSE   module3/   requirements.txt
environment.yml      module1/  module4/   sample_data/
kaggleChallenge.csv  module2/  README.md
